In [120]:
import numpy as np
import matplotlib.pyplot as plt
import options_trading as opt
import pandas as pd
import yfinance as yf
from scipy.optimize import minimize
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Symboles de Futures contracts sur yfinance
##### -----------------------------------------
##### S&P 500 E-Mini Futures : "ES=F"
##### Nasdaq-100 E-Mini Futures : "NQ=F"
##### Pétrole Brut (WTI) : "CL=F"
##### Or : "GC=F"
##### Argent : "SI=F"

### Historical Data

In [121]:
share=yf.download('AAPL', start='2024-01-01')
share.dropna(inplace=True)
share.ffill(inplace=True)
share['Log Return']=np.log(share['Adj Close']/share['Adj Close'].shift(1))
share_var=share['Log Return'].std()*np.sqrt(252)
share_var

[*********************100%%**********************]  1 of 1 completed


0.23430236799723275

In [122]:
futures=yf.download("ES=F", start="2024-01-01")
futures.dropna(inplace=True)
futures.ffill(inplace=True)
futures['Log Return']=np.log(futures['Adj Close']/futures['Adj Close'].shift(1))
futures_var=futures['Log Return'].std()*np.sqrt(252)
futures_var

[*********************100%%**********************]  1 of 1 completed


0.12744338623052165

In [123]:
rho=share['Log Return'].corr(futures['Log Return'])
rho

0.5422462009476742

In [124]:
h = rho*(share_var/futures_var)
h

0.9969098646651939

In [125]:
### VA : amount in asset A
### VF : amount in futures contracts
### Number of contracts to hedge

VA=100
VF=120

N=h*(VA/VF)
N

0.8307582205543284

In [128]:
def cross_hedge (share_ticker, futures_ticker, QA, QF, start_date = "2024-01-01"):
    """
    share_ticker : ticker of share used to hedge the futures
    futures_ticker : ticker of the futures to hedge
    QA : size of position being hedged
    QF : size of futures contract
    """
    ### Compute share variance
    share=yf.download(share_ticker, start=start_date)
    share.dropna(inplace=True)
    share.ffill(inplace=True)
    share['Log Return']=np.log(share['Adj Close']/share['Adj Close'].shift(1))
    share_var=share['Log Return'].std()*np.sqrt(252)
    share_price = share['Adj Close'].iloc[-1]

    ### Compute futures variance
    futures=yf.download(futures_ticker, start="2024-01-01")
    futures.dropna(inplace=True)
    futures.ffill(inplace=True)
    futures['Log Return']=np.log(futures['Adj Close']/futures['Adj Close'].shift(1))
    futures_var=futures['Log Return'].std()*np.sqrt(252)
    futures_price = futures['Adj Close'].iloc[-1]


    ### Compute correlation between two assets
    rho=share['Log Return'].corr(futures['Log Return'])

    ### Compute minimum variance hedge ratio
    h = rho*(share_var/futures_var)
    print(h)

    VA = share_price*QA
    VF = futures_price*QF

    ### Compute the number of contracts to hedge the futures and lock the price
    N=h*(VA/VF)

    return round(N)

In [156]:
cross_hedge(share_ticker='AAPL', futures_ticker='ES=F', QA=1000000, QF=10000)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

0.9969097788485114


4

### Build a portfolio to hedge a futures

In [158]:
### The goal is to replicate the futures price in order to hedge the futures
def get_normalized_series(series):
    return series / series.iloc[0]

def get_portfolio_value(df, weights):
    value=pd.Series(0, index=df.index)
    for i in range (df.shape[1]):
        normalized_prices = get_normalized_series(df.iloc[:, i])
        value+=weights[i]*normalized_prices
    return value

def replicate_portfolio (df, weights, futures_prices):
    futures_prices_normalized = get_normalized_series(futures_prices)
    portfolio_value = get_portfolio_value(df, weights)
    aligned_portfolio, aligned_futures = portfolio_value.align(futures_prices_normalized, join='inner')
    return abs(aligned_portfolio-aligned_futures)

def minimize_portfolio(df, futures_prices):
    """

    """
    n=df.shape[1]
    init_guess = np.repeat(1/n, n)
    bounds = ((0.0, 1.0),)*n

    def objective(weights):
        replication_error = replicate_portfolio(df, weights, futures_prices)
        return replication_error.sum()

    results = minimize(objective, init_guess, method='SLSQP', options={'disp' : False}, bounds=bounds)
    return results.x

def optimal_weights_linear(df, futures_prices):
    futures_prices_normalized = get_normalized_series(futures_prices)
    X = df.apply(get_normalized_series)
    weights, residuals, rank, s = np.linalg.lstsq(X, futures_prices_normalized, rcond=None)
    return weights

### Use to denormalized values and give the final replicated portfolio
def get_real_portfolio_value(df, weights, prices):
    value=pd.Series(0, index=df.index)
    for i in range (df.shape[1]):
        value+=weights[i]*df.iloc[:, i]*(prices.iloc[0]/df.iloc[0, i])
    return value

In [162]:
def get_replicated_portfolio(tickers_list, futures_ticker, start_date="2024-01-01", method="minimizer"):
    """
    method : "minimizer" or "linear_optimizer"
    """

    ### Get futures prices
    futures_prices = yf.download(futures_ticker, start=start_date, progress=False)['Adj Close'].rename(f"Futures {futures_ticker}")
    

    ### Get prices of all shares used to replicate futures prices
    df=pd.DataFrame()
    for ticker in tickers_list:
        share_prices = yf.download(ticker, start=start_date, progress=False)['Adj Close'].rename(ticker)
        df=pd.concat([df, share_prices], axis=1)
    
    ### Get the optimal weights for the replicated portfolio
    if method=="minimizer":
        optimal_weights=minimize_portfolio(df, futures_prices)
    elif method=="linear_optimizer":
        optimal_weights=optimal_weights_linear(df, futures_prices)
    else:
        print("method should be 'minimizer' or 'linear_optimizer'")

    ### Use these weights and denormalized the values
    final_portfolio = get_real_portfolio_value(df, optimal_weights, futures_prices)

    return final_portfolio

In [163]:
tickers_list = ['AAPL', 'GOOGL', 'AMZN', 'MSFT']
futures_tickers = 'ES=F'

portfolio=get_replicated_portfolio(tickers_list, futures_tickers)
portfolio

2024-01-02    4707.056567
2024-01-03    4674.762961
2024-01-04    4596.092024
2024-01-05    4596.931413
2024-01-08    4706.844257
                 ...     
2024-10-28    5818.357601
2024-10-29    5866.810819
2024-10-30    5857.922269
2024-10-31    5663.455301
2024-11-01    5790.147762
Length: 212, dtype: float64

In [164]:
### Then use this portfolio as an asset to cross hedge the futures !

def cross_hedge_v2 (tickers_list, futures_ticker, QA, QF, start_date = "2024-01-01"):
    """
    share_ticker : ticker of share used to hedge the futures
    futures_ticker : ticker of the futures to hedge
    QA : size of position being hedged
    QF : size of futures contract
    """
    ### Get the replicated portfolio
    portfolio=get_replicated_portfolio(tickers_list, futures_tickers)
    returns=np.log(portfolio/portfolio.shift(1))

    ### Compute the variance of the portfolio
    portfolio_var=returns.std()*np.sqrt(252)
    portfolio_price=portfolio.iloc[-1]

    ### Compute futures variance
    futures=yf.download(futures_ticker, start="2024-01-01")
    futures.dropna(inplace=True)
    futures.ffill(inplace=True)
    futures['Log Return']=np.log(futures['Adj Close']/futures['Adj Close'].shift(1))
    futures_var=futures['Log Return'].std()*np.sqrt(252)
    futures_price = futures['Adj Close'].iloc[-1]


    ### Compute correlation between two assets
    rho=returns.corr(futures['Log Return'])

    ### Compute minimum variance hedge ratio
    h = rho*(portfolio_var/futures_var)
    print(h)

    VA = portfolio_price*QA
    VF = futures_price*QF

    ### Compute the number of contracts to hedge the futures and lock the price
    N=h*(VA/VF)

    return round(N)

In [165]:
cross_hedge_v2(tickers_list, futures_tickers, QA=10000, QF=10000)

[*********************100%%**********************]  1 of 1 completed

1.2240652544337634


1